In [5]:
from StartingCandidateGenerator import StartingCandidateGenerator as SCG
from DataBaseInterface import DataBaseInterface as DBI
from CrossOperation import CrossOperation as CO
"Import necessary to fix atoms in the slab"
from ase.constraints import FixAtoms
from ase import Atoms
from ase.io import read
from generate_ceo2 import create_ceo2_111 as ceo2Slab
import numpy as np
from ase.visualize import view
from ase.db import connect
from pathlib import Path
from ase.ga.startgenerator import StartGenerator

from ase.optimize import BFGS
from ase.calculators.emt import EMT

#--------------------------Define the fitness fucntion for your atoms-----------------------"

def fitness_function(atoms,env)-> float:
    o2_energy = read('test_data/o2_pw91.traj').get_potential_energy()
    reference=0.0# read('test_data/pt6o0_fs_meta.traj@:')[0].get_potential_energy()

    at_num= atoms.get_atomic_numbers()
    o_num = np.count_nonzero(at_num == 8)
    ce_num= np.count_nonzero(at_num == 58)

    o_num = o_num - (2* ce_num)
    fre = atoms.get_potential_energy() - reference - o_num * (0.5 * o2_energy) - o_num*env*0.5
    return fre

def relax_candidate(dbid,calculator):
    atoms = db.get_atoms_from_id(dbid)
    
    atoms.calc = calculator
    atoms.get_potential_energy()

    atoms.info['key_value_pairs']['raw_score'] = fitness_function(atoms,env)
    db.update_to_relaxed(dbid,atoms)

#---------------------------Generate static part of the system------------------------------"
slab = ceo2Slab(number_of_layers=1,repetitions = (5,5,1),vacuum=12.0)
a = 12.0
slab = Atoms('C',cell=[a,a,a],
             pbc=True)
c = FixAtoms(indices=np.arange(len(slab)))
slab.set_constraint(c)
#If slab atoms arent fixed they will be generated  for all structures, but will be relaxed!"

#---------------------------Generate constant part of the system------------------------------"
#Part of the system that can be relaxed but that doe snot vary in number over the duration of the search"
constant = Atoms('Pt6')

#---------Generate variable part of the system-(Single atom types only)------------------"
#Part of the system that can be relaxed and that varyies in number over the duration of the search"
#
variable_type = Atoms('O')
#Considered number of variable type atoms n the search"
variable_range = [0,1,2,3,4,7,8,9,10]

#---------------------------Define starting populatio--------------------------------"

candidateGenerator = SCG(slab,constant,variable_type,variable_range)
crossing = CO(slab,constant,variable_type,variable_range,stc_change_chance = 0.5)
db = DBI('databaseGA.db')
population = 25

starting_pop = candidateGenerator.get_starting_population(population_size=population)



for i in starting_pop:
    db.add_unrelaxed_candidate(i )

        
#-----------------------------------Define Calculator----------------------------------------"

calc = EMT()

#---------------------------------Relax initial structures-----------------------------------"
env = -0.5 #Environment chem pot ox
while db.get_number_of_unrelaxed_candidates() > 0:

    atoms = db.get_first_unrelaxed()
    
    atoms.calc = calc
    dyn = BFGS(atoms)
    dyn.run(steps=100, fmax=0.05)
    atoms.get_potential_energy()
    
    atoms.info['key_value_pairs']['raw_score'] = -fitness_function(atoms,env)
   
    db.update_to_relaxed(atoms.info['key_value_pairs']['dbid'],atoms)

#--------------------------------------Find better stoich to srtart eval----------------------------
#Overall fitness value
steps = 10
sub_steps = 20
for i in range(steps):
    for j in range(sub_steps):
        atomslist = db.get_better_candidates(n=2)
        "Mutate these atoms"
        res  = crossing.cross(atomslist[0],atomslist[1])
        if(res is not None):
            db.add_unrelaxed_candidate(res)
    
    while db.get_number_of_unrelaxed_candidates() > 0:

        atoms = db.get_first_unrelaxed()

        atoms.calc = calc
        dyn = BFGS(atoms)
        dyn.run(steps=100, fmax=0.05)
        atoms.get_potential_energy()

        atoms.info['key_value_pairs']['raw_score'] = -fitness_function(atoms,env)

        db.update_to_relaxed(atoms.info['key_value_pairs']['dbid'],atoms)
        
atomslist = db.get_better_candidates(n=2)
for at in atomslist:
    view (at)

      Step     Time          Energy         fmax
BFGS:    0 17:44:25       19.293926       22.6773
BFGS:    1 17:44:25       14.624914        5.5739
BFGS:    2 17:44:25       13.444105        2.9085
BFGS:    3 17:44:25       13.262744        1.8339
BFGS:    4 17:44:25       13.201682        2.2459
BFGS:    5 17:44:25       13.148070        0.7258
BFGS:    6 17:44:25       13.115281        0.7944
BFGS:    7 17:44:25       12.706033        2.6602
BFGS:    8 17:44:25       12.652122        5.5849
BFGS:    9 17:44:25       12.556376        3.1744
BFGS:   10 17:44:25       12.501331        3.4419
BFGS:   11 17:44:25       12.474110        3.7811
BFGS:   12 17:44:25       12.436864        3.3798
BFGS:   13 17:44:25       12.408098        3.0316
BFGS:   14 17:44:25       12.300455        1.8486
BFGS:   15 17:44:25       12.086180        1.7316
BFGS:   16 17:44:25       12.304548        3.0177
BFGS:   17 17:44:25       12.048228        1.4683
BFGS:   18 17:44:25       12.026819        1.5825
B

BFGS:   31 17:44:25       11.228844        0.1799
BFGS:   32 17:44:25       11.222518        0.2681
BFGS:   33 17:44:25       11.207369        0.5651
BFGS:   34 17:44:25       11.180958        1.0441
BFGS:   35 17:44:25       11.154889        0.9247
BFGS:   36 17:44:25       11.069009        0.5076
BFGS:   37 17:44:26       11.263698        3.1436
BFGS:   38 17:44:26       11.032918        0.3985
BFGS:   39 17:44:26       11.011429        0.4012
BFGS:   40 17:44:26       10.941045        1.2326
BFGS:   41 17:44:26       10.908431        0.8040
BFGS:   42 17:44:26       10.822304        0.8323
BFGS:   43 17:44:26       10.789321        0.5030
BFGS:   44 17:44:26       10.763174        0.4235
BFGS:   45 17:44:26       10.756609        0.5277
BFGS:   46 17:44:26       10.745436        0.4359
BFGS:   47 17:44:26       10.733616        0.2570
BFGS:   48 17:44:26       10.726008        0.2558
BFGS:   49 17:44:26       10.720139        0.2517
BFGS:   50 17:44:26       10.714771        0.1894


BFGS:   19 17:44:26       11.171833        3.9691
BFGS:   20 17:44:26       10.955866        1.8920
BFGS:   21 17:44:26       10.882401        1.3710
BFGS:   22 17:44:26       10.824154        0.4721
BFGS:   23 17:44:26       10.815279        0.1037
BFGS:   24 17:44:26       10.814895        0.0450
      Step     Time          Energy         fmax
BFGS:    0 17:44:26       20.384822       14.0449
BFGS:    1 17:44:26       17.809334        3.4607
BFGS:    2 17:44:26       16.820534        3.4787
BFGS:    3 17:44:26       15.407887        7.3802
BFGS:    4 17:44:26       14.867843        1.5395
BFGS:    5 17:44:26       14.758889        1.2990
BFGS:    6 17:44:26       14.691502        1.3151
BFGS:    7 17:44:26       14.469010        2.8480
BFGS:    8 17:44:26       14.058332        9.3656
BFGS:    9 17:44:26       14.759655       15.6833
BFGS:   10 17:44:26       13.540886        6.9908
BFGS:   11 17:44:26       13.236450        5.0462
BFGS:   12 17:44:26       12.868442        2.4771
B

BFGS:   55 17:44:27        9.298225        0.6948
BFGS:   56 17:44:27        9.246636        0.5864
BFGS:   57 17:44:27        9.215317        0.4568
BFGS:   58 17:44:27        9.197809        0.3267
BFGS:   59 17:44:27        9.188149        0.3792
BFGS:   60 17:44:27        9.183874        0.2472
BFGS:   61 17:44:27        9.180166        0.2261
BFGS:   62 17:44:27        9.173888        0.2991
BFGS:   63 17:44:27        9.165682        0.3795
BFGS:   64 17:44:27        9.157857        0.3826
BFGS:   65 17:44:27        9.151272        0.2964
BFGS:   66 17:44:27        9.143893        0.2407
BFGS:   67 17:44:27        9.137288        0.2246
BFGS:   68 17:44:27        9.130143        0.2715
BFGS:   69 17:44:27        9.126307        0.3318
BFGS:   70 17:44:27        9.124097        0.2564
BFGS:   71 17:44:27        9.121403        0.2202
BFGS:   72 17:44:27        9.117816        0.2079
BFGS:   73 17:44:27        9.115185        0.1791
BFGS:   74 17:44:27        9.113139        0.1708


BFGS:   33 17:44:28       14.806693        0.2757
BFGS:   34 17:44:28       14.788551        0.4479
BFGS:   35 17:44:28       14.720185        1.0336
BFGS:   36 17:44:28       14.622946        1.7059
BFGS:   37 17:44:28       14.982921        3.1079
BFGS:   38 17:44:28       14.602846        1.7237
BFGS:   39 17:44:28       14.582726        1.7064
BFGS:   40 17:44:28       14.669804        3.2564
BFGS:   41 17:44:28       14.470826        1.3197
BFGS:   42 17:44:28       14.430900        1.0033
BFGS:   43 17:44:28       14.408359        0.7103
BFGS:   44 17:44:28       14.394282        0.4084
BFGS:   45 17:44:28       14.389018        0.3662
BFGS:   46 17:44:28       14.382258        0.2223
BFGS:   47 17:44:28       14.381231        0.2186
BFGS:   48 17:44:28       14.373089        0.3158
BFGS:   49 17:44:28       14.364960        0.3730
BFGS:   50 17:44:28       14.351305        0.5012
BFGS:   51 17:44:28       14.334774        0.4291
BFGS:   52 17:44:28       14.314616        0.3454


...

BFGS:   39 17:45:24        5.571533        0.0746
BFGS:   40 17:45:24        5.571206        0.0445
      Step     Time          Energy         fmax
BFGS:    0 17:45:24        8.848978        0.2813
BFGS:    1 17:45:24        8.846367        0.1842
BFGS:    2 17:45:24        8.844307        0.2112
BFGS:    3 17:45:24        8.842879        0.1973
BFGS:    4 17:45:24        8.840145        0.2015
BFGS:    5 17:45:24        8.837694        0.1923
BFGS:    6 17:45:24        8.835032        0.1756
BFGS:    7 17:45:24        8.832712        0.2333
BFGS:    8 17:45:24        8.829676        0.2633
BFGS:    9 17:45:24        8.824123        0.3022
BFGS:   10 17:45:24        8.810687        0.5778
BFGS:   11 17:45:24        8.774625        1.0295
BFGS:   12 17:45:24        8.732397        1.3536
BFGS:   13 17:45:24        8.641952        1.4002
BFGS:   14 17:45:24        8.518790        1.3194
BFGS:   15 17:45:24        8.363435        1.1829
BFGS:   16 17:45:24        8.176941        1.2071
B

BFGS:   32 17:45:28        6.017142        0.6400
BFGS:   33 17:45:28        5.971619        0.5795
BFGS:   34 17:45:28        5.926734        0.5598
BFGS:   35 17:45:28        5.894468        0.5720
BFGS:   36 17:45:28        5.873824        0.5605
BFGS:   37 17:45:28        5.860036        0.5739
BFGS:   38 17:45:28        5.836422        0.6748
BFGS:   39 17:45:28        5.813317        0.5084
BFGS:   40 17:45:28        5.800170        0.2407
BFGS:   41 17:45:28        5.797168        0.2126
BFGS:   42 17:45:28        5.795656        0.1931
BFGS:   43 17:45:28        5.793269        0.1129
BFGS:   44 17:45:28        5.792701        0.0816
BFGS:   45 17:45:28        5.792314        0.0555
BFGS:   46 17:45:28        5.792097        0.0555
BFGS:   47 17:45:28        5.791889        0.0543
BFGS:   48 17:45:28        5.791684        0.0555
BFGS:   49 17:45:28        5.791533        0.0472
      Step     Time          Energy         fmax
BFGS:    0 17:45:28        8.975458        0.2814
B

BFGS:   24 17:45:29        8.409345        0.4654
BFGS:   25 17:45:29        8.398185        0.3017
BFGS:   26 17:45:29        8.392455        0.1516
BFGS:   27 17:45:29        8.391634        0.0749
BFGS:   28 17:45:29        8.391499        0.0268
      Step     Time          Energy         fmax
BFGS:    0 17:45:29        8.935864        0.2710
BFGS:    1 17:45:29        8.933313        0.1574
BFGS:    2 17:45:29        8.931894        0.1363
BFGS:    3 17:45:29        8.931498        0.0963
BFGS:    4 17:45:29        8.930891        0.0833
BFGS:    5 17:45:29        8.930404        0.0958
BFGS:    6 17:45:29        8.929616        0.0840
BFGS:    7 17:45:29        8.929028        0.0928
BFGS:    8 17:45:29        8.928552        0.1070
BFGS:    9 17:45:29        8.927927        0.1082
BFGS:   10 17:45:29        8.926475        0.1761
BFGS:   11 17:45:29        8.922709        0.3091
BFGS:   12 17:45:29        8.910873        0.5970
BFGS:   13 17:45:29        8.888761        0.8622
B

BFGS:    6 17:45:29        8.592540        0.4175
BFGS:    7 17:45:29        8.566965        0.5916
BFGS:    8 17:45:29        8.521348        0.8423
BFGS:    9 17:45:29        8.421971        1.0299
BFGS:   10 17:45:29        8.233421        1.1134
BFGS:   11 17:45:29        7.938980        1.5430
BFGS:   12 17:45:29        7.546322        1.9468
BFGS:   13 17:45:29        7.105367        1.9746
BFGS:   14 17:45:29        6.746023        1.1161
BFGS:   15 17:45:29        6.709370        1.8227
BFGS:   16 17:45:29        6.618864        1.0402
BFGS:   17 17:45:29        6.575651        0.9640
BFGS:   18 17:45:29        6.440282        2.4870
BFGS:   19 17:45:29        6.345467        0.6320
BFGS:   20 17:45:30        6.306579        0.6238
BFGS:   21 17:45:30        6.247149        1.0598
BFGS:   22 17:45:30        6.214320        0.5016
BFGS:   23 17:45:30        6.198840        0.5159
BFGS:   24 17:45:30        6.137090        0.5573
BFGS:   25 17:45:30        6.070100        0.8066


BFGS:   25 17:45:30        5.475881        0.4091
BFGS:   26 17:45:30        5.459255        0.5246
BFGS:   27 17:45:30        5.440838        0.4644
BFGS:   28 17:45:30        5.451217        0.6825
BFGS:   29 17:45:30        5.414426        0.2151
BFGS:   30 17:45:30        5.409949        0.2080
BFGS:   31 17:45:30        5.406088        0.1689
BFGS:   32 17:45:30        5.404478        0.1735
BFGS:   33 17:45:30        5.402722        0.1193
BFGS:   34 17:45:30        5.401885        0.0629
BFGS:   35 17:45:30        5.401482        0.0531
BFGS:   36 17:45:30        5.401183        0.0436
